<a href="https://colab.research.google.com/github/M-Shrock/RankedVoting/blob/main/RankVote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rcv

In [18]:
#authenticate with google, import spread sheet, and create initial DataFrame
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

#open spread sheet producet from google form
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eueZkL35Z-57CBagal3aWgMMEKyVZSfzJx6UrmfEf9M/edit#gid=1069421963')

#get list of all rows in rows
wb = wb.worksheet('Form Responses 1')
rows = wb.get_all_values()

#convert to DataFrame and render
import pandas as pd
df = pd.DataFrame.from_records(rows)

#print(df)

In [19]:
#format DataFrame and clean data

#trim the header row
new_header = df.iloc[0]
df = df[1:]

#trim column 1 (timestamp), column 2 (email), and column 10 (suggestion/comments)
df.columns = new_header
df = df.iloc[:,2:10]

#schedule options and labels for parallel indexing (ts: time slot)
options = ["Monday 4-6pm", "Wednesday 4-6pm", "Friday 4-6pm", "Monday 12-2pm", "Tuesday 12-2pm", "Thursday 12-2pm", "Friday 12-2pm", "Tuesday 5-6pm"]
optionsLabel = ["ts1", "ts2", "ts3", "ts4", "ts5", "ts6", "ts7", "ts8"]
OPTIONS_SIZE = len(options)

#interate thru options, replace val for i in df
for i,val in enumerate(options):
  df.replace(options[i], optionsLabel[i], True)

#clean up column names
col_header = ["1st", "2nd", "3rd", "4th", "5th", "6th", "7th", "8th"]
df.columns = col_header
df = df.reset_index().iloc[:,1:]

#print(df)

In [20]:
#employ rcv library for ranked choice - single transferable vote 
#https://pypi.org/project/rcv/#:~:text=rcv%20is%20a%20Python%20library,the%20BSD%203%2DClause%20License.
from rcv import FractionalSTV, PreferenceSchedule

#create schedule from row data of df and calculate winners
schedule = PreferenceSchedule.from_dataframe(df)
stv = FractionalSTV(schedule, seats=2)
winners = stv.elect()

for i,val in enumerate(optionsLabel):
  if val in winners:
    print(options[i])

Monday 4-6pm
Monday 12-2pm
